# Cross Validation Lab

In this notebook, I am going to use the mtcars dataset as a worked example. You are to read this, try running the cells and understand what is going on in them. Get the logic behind the things I've tried

1) Looking at train_test_split and comparing training scores to test scores.
2) Getting cross validation scores to make a selection on which features to use.
3) Trying Polynomial Regression on a variable to see if it can improve things.

After this, I want you to repeat the procedure with the Diabetes dataset

First of all, let's look at the mtcars (1974 Motor Trend Cars) dataset and see what we can do with that.
<br><img src="images/mtcars.png" width="500"/><br>
Let's try more than one feature and see if this improves the model. We should be using separate sets for training and evaluating so let's try that now and see what happens

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#load mtcars
dfcars=pd.read_csv("week02\mtcars.csv")
dfcars=dfcars.rename(columns={"Unnamed: 0":"name"})
dfcars.head()

y is the response variable so let's slice that off

In [3]:
y = dfcars['mpg']

We have multiple possible X's let's put them aside too, we don't want the first two columns

In [4]:
allX = dfcars.iloc[:, 2:]

In [ ]:
allX.head()

We first of all need to separate the data into a training set and a test set. We can use train_test_split for this

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
allX_train, allX_test, y_train, y_test = train_test_split(allX, y, test_size=0.2, random_state=42)

I did random_state=42 so the answer is the same everytime I run this workbook.

Ok now we have two different datasets, train and test that we can use where appropriate. First of all, we just want to try different features

I think wt would be a good feature to select so I'll go with that

Separating out just that column into what we are going to fit

In [8]:
X_train = allX_train[["wt"]]
X_test = allX_test[["wt"]]

Now build the model

In [9]:
from sklearn.linear_model import LinearRegression

In [ ]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

Let's look at the scores of both the training set and test set

In [ ]:
model1.score(X_train, y_train)

In [ ]:
model1.score(X_test, y_test)

There is a decent difference between the scores so maybe overfit a little. Anyway let's try a different feature

In [13]:
X_train = allX_train[["hp"]]
X_test = allX_test[["hp"]]

In [ ]:
model2 = LinearRegression()
model2.fit(X_train, y_train)

In [ ]:
model2.score(X_train, y_train)

In [ ]:
model2.score(X_test, y_test)

Ok hp is even worse performing than wt. Not a good one to use

drat?

In [17]:
X_train = allX_train[["drat"]]
X_test = allX_test[["drat"]]

In [ ]:
model3 = LinearRegression()
model3.fit(X_train, y_train)

In [ ]:
model3.score(X_train, y_train)

In [ ]:
model3.score(X_test, y_test)

The best I tried was wt, the first one. Anyway let's try two variables

In [21]:
X_train = allX_train[["wt","hp"]]
X_test = allX_test[["wt","hp"]]

In [ ]:
model4 = LinearRegression()
model4.fit(X_train, y_train)

In [ ]:
model4.score(X_train, y_train)

That is a better training score, but what about the test score which is really what's important

In [ ]:
model4.score(X_test, y_test)

That's the best we've had so far, so model4 is the best one of that

## Validation
Hold on, I've contradicted a little what I said in the "lecture". I'm using the test score to make decisions. I should not be doing this. I should use some form of validation. So here's an example of that

In [25]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score?

Let's try wt again

In [27]:
X_train = allX_train[["wt"]]
X_test = allX_test[["wt"]]

In [ ]:
model5 = LinearRegression()
scores = cross_val_score(model5, X_train, y_train)
scores

In [ ]:
scores.mean()

Now let's try the two variables, wt and hp again

In [30]:
X_train = allX_train[["wt","hp"]]
X_test = allX_test[["wt","hp"]]

In [ ]:
model6 = LinearRegression()
scores6 = cross_val_score(model5, X_train, y_train)
scores6

In [ ]:
scores6.mean()

So the two together has a higher cross validation score that just one. This means we should select the two variables over one 

What about 3? Let's try drat

In [33]:
X_train = allX_train[["wt","hp","drat"]]
X_test = allX_test[["wt","hp","drat"]]

In [ ]:
model7 = LinearRegression()
scores7 = cross_val_score(model5, X_train, y_train)
scores7.mean()

Slightly better than 2. It is not always the case that more features the better, just happened in the above example

Let's try three features again, but this time with carb instead of drat

In [35]:
X_train = allX_train[["wt","hp","carb"]]
X_test = allX_test[["wt","hp","carb"]]

In [ ]:
model8 = LinearRegression()
scores8 = cross_val_score(model5, X_train, y_train)
scores8.mean()

This score is worse than only using the two features wt and hp

Our best so far (according to the cross val scores) has been model7, the one with ["wt","hp","drat"]. So let's build that model fully and then evaluate that

In [ ]:
X_train = allX_train[["wt","hp","drat"]]
X_test = allX_test[["wt","hp","drat"]]
model7 = LinearRegression()
model7.fit(X_train, y_train)
model7.score(X_test, y_test)

A score of .79 for the test set

## Polynomial Regression Example

I am going to use the 'wt' as a feature and see what happens with a polynomial regression model

Some notes about Polynomial and sklearn

### The `scikit-learn` interface

Scikit-learn is the main python machine learning library. It consists of many learners which can learn models from data, as well as a lot of utility functions such as `train_test_split`. It can be used in python by the incantation `import sklearn`.

The library has a very well defined interface. This makes the library a joy to use, and surely contributes to its popularity. As the [scikit-learn API paper](http://arxiv.org/pdf/1309.0238v1.pdf) [Buitinck, Lars, et al. "API design for machine learning software: experiences from the scikit-learn project." arXiv preprint arXiv:1309.0238 (2013).] says:

>All objects within scikit-learn share a uniform common basic API consisting of three complementary interfaces: **an estimator interface for building and ﬁtting models, a predictor interface for making predictions and a transformer interface for converting data**. The estimator interface is at the core of the library. It deﬁnes instantiation mechanisms of objects and exposes a `fit` method for learning a model from training data. All supervised and unsupervised learning algorithms (e.g., for classiﬁcation, regression or clustering) are oﬀered as objects implementing this interface. Machine learning tasks like feature extraction, feature selection or dimensionality reduction are also provided as estimators.

We'll use the "estimator" interface here, specifically the estimator `PolynomialFeatures`. The API paper again:

>Since it is common to modify or ﬁlter data before feeding it to a learning algorithm, some estimators in the library implement a transformer interface which deﬁnes a transform method. It takes as input some new data X and yields as output a transformed version of X. Preprocessing, feature selection, feature extraction and dimensionality reduction algorithms are all provided as transformers within the library.

To start with we have one **feature** `x` to predict `y`, what we will do is the transformation:

$$ x \rightarrow 1, x, x^2, x^3, ..., x^d $$

for some power $d$. Our job then is to **fit** for the coefficients of these features in the polynomial

$$ a_0 + a_1 x + a_2 x^2 + ... + a_d x^d. $$

In other words, we have transformed a function of one feature, into a (rather simple) **linear** function of many features. To do this we first construct the estimator as `PolynomialFeatures(d)`, and then transform these features into a d-dimensional space using the method `fit_transform`.

![fit_transform](images/sklearntrans.jpg)

Here is an example. The reason for using `[[1],[2],[3]]` as opposed to `[1,2,3]` is that scikit-learn expects data to be stored in a two-dimensional array or matrix with size `[n_samples, n_features]`.

To transform `[1,2,3]` into [[1],[2],[3]] we need to do a reshape.

![reshape](images/reshape.jpg)

In [ ]:
demo = np.array([1,2,3]).reshape(-1,1)
demo

In [39]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
PolynomialFeatures(5).fit_transform(demo)

This is $x^0 , x^1, x^2, x^3, x^4, x^5$

In [41]:
X_train = allX_train[["wt"]]
X_test = allX_test[["wt"]]

Let's try it with PolynomialFeatures(2)

In [42]:
X_train_poly = PolynomialFeatures(2).fit_transform(X_train)
X_test_poly = PolynomialFeatures(2).fit_transform(X_test)

In [ ]:
modelpoly2 = LinearRegression()
modelpoly2.fit(X_train_poly, y_train)

In [ ]:
print(modelpoly2.score(X_train_poly, y_train))
print(modelpoly2.score(X_test_poly, y_test))

Let's now do a loop for more of them and record all the scores in an array

We will initialise the array as all zeros

In [45]:
from sklearn.metrics import mean_squared_error

In [46]:
max_p = 10
degrees = range(max_p+1)
train_scores = np.zeros(max_p+1)
test_scores = np.zeros(max_p+1)
error_train = np.zeros(max_p+1)
error_test = np.zeros(max_p+1)

In [47]:
for i in degrees:
    if i != 0:
        X_train_poly = PolynomialFeatures(i).fit_transform(X_train)
        X_test_poly = PolynomialFeatures(i).fit_transform(X_test)
        polymodel = LinearRegression()
        polymodel.fit(X_train_poly, y_train)
        prediction_on_training = polymodel.predict(X_train_poly)
        prediction_on_test = polymodel.predict(X_test_poly)
        error_train[i] = mean_squared_error(y_train, prediction_on_training)
        error_test[i] = mean_squared_error(y_test, prediction_on_test)
        train_scores[i] = polymodel.score(X_train_poly, y_train)
        test_scores[i] = polymodel.score(X_test_poly, y_test)

In [ ]:
train_scores

Now let's look at plots and things. I am going to keep doing error_train[1:], error_test[1:] so the 0 value is not counted

This will give us the degree with the lowest error in the test set

In [49]:
bestd = np.argmin(error_test[1:])+1

In [ ]:
plt.plot(degrees[1:], error_train[1:], marker='o', label='train (in-sample)')
plt.plot(degrees[1:], error_test[1:], marker='o', label='test')
plt.axvline(bestd, 0,0.5, color='g', label="min test error at d=%d"%bestd, alpha=0.3)
plt.ylabel('mean squared error')
plt.xlabel('degree')
plt.legend(loc='upper left')
plt.yscale("log")

So the simple linear regression $y = w_0 + w_1x$ works best for wt

Notice how the training error keeps getting smaller as the degree increases, but this does not correspond to a better test error

This is expected if we looked at the scatter plot of wt vs mpg

In [ ]:
plt.scatter(dfcars["wt"],dfcars["mpg"])

Relationship looks linear

## Some work for you to do

## Firstly - Validation


What we have done in picking a given degree $d$ as the best hypothesis is that we have used the test set as a training set. 
If we choose the best $d$ based on minimizing the test set error, we have then "fit for" hyperparameter $d$ on the test set. 

In this case, the test-set error will underestimate the true out of sample error. Furthermore, we have **contaminated the test set** by fitting for $d$ on it; it is no longer a true test set.

Thus, we introduce a new **validation set** on which the complexity parameter $d$ is fit, and leave out a test set which we can use to estimate the true out-of-sample performance of our learner. The place of this set in the scheme of things is shown below:

![m:caption](images/train-validate-test.png)

We have split the old training set into a **new smaller training set** and a **validation set**, holding the old test aside for FINAL testing AFTER we have "fit" for complexity $d$. Obviously we have decreased the size of the data available for training further, but this is a price we must pay for obtaining a good estimate of the out-of-sample error 

![m:caption](images/train-validate-test-cont.png)

The validation process is illustrated in these two figures. We first loop over the complexity parameter $d$, the degree of the polynomials we will try and fit. Then for each degree $d$, we obtain a best fit model $g^-_d$ where the "minus" superscript indicates that we fit our model on the new training set which is obtained by removing ("minusing") a validation chunk (often the same size as the test chunk) from the old training set. We then "test" this model on the validation chunk, obtaining the validation error for the best-fit polynomial coefficients and for degree $d$. We move on to the next degree $d$ and repeat the process, just like before. We compare all the validation set errors, just like we did with the test errors earlier, and pick the degree $d_*$ which minimizes this validation set error.

![caption](images/train-validate-test3.png)

Having picked the hyperparameter $d_*$, we retrain on the entire old training-set to find the parameters of the polynomial of order $d_*$.  We now compute the test error on the test set as an estimate of the test error.

Thus the **validation** set is the set on which the hyperparameter is fit. This method of splitting the data $\cal{D}$ is called the **train-validate-test** split.

### Fit on training and predict on validation


We carry out this process for one training/validation split below. Note the smaller size of the new training set. We hold the test set at the same size.

Firstly, let's split the training set up further into X_v_train, X_v_valid, y_v_train and y_v_valid using train_test_split again

In [52]:
# your code here
X_v_train, X_v_valid, y_v_train, y_v_valid = train_test_split(X_train, y_train)

In [ ]:
X_v_train.shape


>YOUR TURN HERE: Train on the smaller training set. Fit for d on the validation set.  Store the respective MSEs in `error_train` and `error_valid`. Then retrain on the entire training set using this d. Label the test set MSE with the variable `err`.

In [54]:
#Note code below hasn't been tested but is here for psuedocode suggestion

error_train=np.empty(len(degrees))
error_valid=np.empty(len(degrees))
score_train=np.empty(len(degrees))
score_valid=np.empty(len(degrees))
#for each degree, we now fit on the smaller training set and predict on the validation set
#we accumulate the MSE on both sets in error_train and error_valid
#we then find the degree of polynomial that minimizes the MSE on the validation set.
#your code here
for d in degrees:#for increasing polynomial degrees 0,1,2...
    #Create polynomials from X_v_train and X_v_valid
    X_c = PolynomialFeatures(d).fit_transform(X_v_train)
    X_c_val = PolynomialFeatures(d).fit_transform(X_v_valid)
    #fit a model linear in polynomial coefficients on the new smaller training set
    est = LinearRegression()
    est.fit(X_c, y_v_train)    
    #predict on new training and validation sets and calculate mean squared error
    error_train[d] = mean_squared_error(est.predict(X_c), y_v_train)
    error_valid[d] = mean_squared_error(est.predict(X_c_val), y_v_valid)
    score_train[d] = est.score(X_c, y_v_train)
    score_valid[d] = est.score(X_c_val, y_v_valid)
    

Plot the training error and validation error against the degree of the polynomial, and show the test set error at the $d$ which minimizes the validation set error.

Fit on WHOLE training set now. You will need to remake polynomial features on the whole training set. Test on TestSet

Try with Cross Validation

# Try now with the Diabetes Dataset (more samples)

In [ ]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
print(diabetes.DESCR)

## Do similar to what I've done with the diabetes dataset

See if you can find a better combination of columns rather than using all of them

Pick one variable and see if a Polynomial Regression model could improve that one variable